In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import sys

In [2]:
imbd_data = pd.read_csv('final_voting_yeah.csv',index_col=0)

In [3]:
text_data = imbd_data.loc[: , ['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor']]

In [4]:
from collections import Counter

In [5]:
DIRECTOR_W = 4
WRITER_W = 1
ACTOR_W = 4
ACTOR_W_1 = 2

In [6]:
weight_dict = {
    'Director':DIRECTOR_W,
    'Writer':WRITER_W,
    'First_actor':ACTOR_W,
    'Second_actor':ACTOR_W,
    'Third_actor':ACTOR_W_1,
    'Forth_actor':ACTOR_W_1
}

In [7]:
def get_weight(type):
    if type == 'Director':
        return word_count(text_data[type].tolist(), DIRECTOR_W)
    elif type == 'Writer':
        return word_count(text_data[type].tolist(), WRITER_W)
    elif type == 'First_actor' or type == 'Second_actor':
        return word_count(text_data[type].tolist(), ACTOR_W)
    else: return word_count(text_data[type].tolist(), ACTOR_W_1)
    
def word_count(name_list, adj_fct):
    count = Counter()
    for name in name_list:
        if type(name) is str: count[name.strip()] += 1 * adj_fct
    return count

def conbime_act():
    c1 = get_weight('First_actor')
    c2 = get_weight('Second_actor')
    c3 = get_weight('Third_actor')
    c4 = get_weight('Forth_actor')
    keys = [*c1.keys()] + [*c2.keys()]  + [*c3.keys()] + [*c4.keys()]   
    m_dic = dict.fromkeys(keys)
    for key in m_dic.keys():
        count = 0
        if key in c1.keys(): count += c1[key]
        if key in c2.keys(): count += c2[key]
        if key in c3.keys(): count += c3[key]
        if key in c4.keys(): count += c4[key]
        m_dic[key] = count
    return m_dic

In [8]:
alpha_smoothing = 0.005 

In [17]:
def normed_adj_weight(name, role):
    soc_loc = ['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']
    col = role
    if role == 'First_actor' or role == 'Second_actor' or role == 'Third_actor' or role == 'Forth_actor':
        col = ['First_actor' , 'Second_actor', 'Third_actor', 'Forth_actor']
        score = 0
        leng = 0
        for co in col:
            idc = text_data[co][text_data[col][text_data[col] == name].notna()[co]].index.values.tolist()
            for id in idc:
                sco_of_film = (imbd_data.iloc[id][['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']]).mean()
                sum_of_weight = sum([weight_dict[item] for item in weight_dict if type(imbd_data.iloc[id][item]) is str])
                base = np.log(sco_of_film)
                score += base + sco_of_film*(weight_dict[co])/sum_of_weight
            leng += len(idc)
        smoothing_weight = (1 + alpha_smoothing)/(leng + len(imbd_data)*alpha_smoothing)
        final_socre = score*smoothing_weight*np.log(len(imbd_data)/conbime_act()[name])
    else: 
        idx = text_data[col][text_data[col] == name].index.values.tolist()
        score = 0
        for id in idx:
            sco_of_film = (imbd_data.iloc[id][['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']]).mean()
            sum_of_weight = sum([weight_dict[item] for item in weight_dict if type(imbd_data.iloc[id][item]) is str])
            base = np.log(sco_of_film)
            score += base + sco_of_film*(weight_dict[role])/sum_of_weight
        smoothing_weight = (1 + alpha_smoothing)/(len(idx) + len(imbd_data)*alpha_smoothing)
        final_socre = score*smoothing_weight*np.log(len(imbd_data)/get_weight(role)[name])
    return final_socre

In [23]:
score_list = []
for i in range(len(imbd_data)):
    score_per_film = 0
    for col in ['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor']:
        if type(imbd_data.iloc[i][col]) is str:
            score_per_film+= normed_adj_weight(imbd_data.iloc[i][col], col)
    score_list.append(score_per_film)

In [24]:
len(score_list)

11116

In [25]:
score_list[:10]

[18.60091103807606,
 20.004369209192493,
 12.257118610848615,
 21.101227328279098,
 21.574697913694727,
 16.534942004304263,
 16.148456328194076,
 11.710897854844657,
 21.200908460159948,
 19.550654657380466]

In [28]:
imbd_data['Adj_Score'] = score_list

In [35]:
my_data = imbd_data.drop(['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor', 'imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic'], axis = 1)

In [ ]:
movie.to_csv('final_version_yeah.csv.csv')